## Langgraph 101

> https://www.youtube.com/watch?v=qaWOwbFw3cs
> https://www.youtube.com/watch?v=PqS1kib7RTw&t=547s

---

Think of langgraph as a giant state machine. The nodes can decide what is current state, what is next state, how to go back, how to exit out of the graph etc. It is more sophisticated than a simple chain

In [ ]:
from typing import TypedDict, Annotated, List, Union
from dotenv import load_dotenv
import os
from loguru import logger
import operator
import random

In [ ]:
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage

In [ ]:
load_dotenv("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env", override=True)

In [ ]:
# print (
#     os.environ['AZURE_API_BASE'],
#     os.environ['AZURE_API_KEY'],
#     os.environ['AZURE_API_VERSION']
# )

In [ ]:
class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: List[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermedeiate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

[Explanation of `AgentState`](https://gemini.google.com/share/f9336ab8b995)

### Custom Tools

**Tools**  

Tools are interfaces that an agent can use to interact with the world. They combine a few things:



1.   The name of the tool
2.   A description of what the tool is
3.   JSON schema of what the inputs to the tool are
4.   The function to call


Whether the result of a tool should be returned directly to the user

In [ ]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [ ]:
@tool("lower_case", return_direct=True)
def to_lower_case(input_str: str) -> str:
    """Returns the input as all lower case."""
    return input_str.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input_str: str) -> int:
    """Returns a random number between 0-100."""

    return random.randint(0, 100)

In [ ]:
random_number_maker.run('random')

In [ ]:
to_lower_case.run('SAM')

### Agent - with new create_open_ai

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import AzureChatOpenAI

In [ ]:
# Get the prompt to use - you can modify this! - https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
prompt.get_prompts()

In [ ]:
chat_gpt4 = AzureChatOpenAI(
        deployment_name="gpt-4-turbo-0125",
        model="gpt-4-turbo-0125",
        openai_api_type="azure",
        azure_endpoint=os.environ['AZURE_API_BASE'],
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version=os.environ['AZURE_API_VERSION'],
        max_retries=2,
        temperature=0,
    )

In [ ]:
chat_gpt4.invoke("hello")

In [ ]:
# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(
    llm=chat_gpt4,
    tools=[to_lower_case,random_number_maker],
    prompt=prompt
)

In [ ]:
type(agent_runnable)

In [ ]:
agent_runnable.input_schema.schema()

In [ ]:
# invoke the agent
inputs = {"input": "give me a random number and then write in words and make it lower case.",
          "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

In [ ]:
agent_outcome

We get a `AgentActionMessageLog` response back

- it selects `random_number` tool via function call

View the logs in the langsmith dashboard

- this has just told us what tool to call, has not called the tool yet

- we will use this as a node in the graph

### Nodes

In [ ]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(
    tools=[to_lower_case, random_number_maker]
)

In [ ]:
# Define the agent/graph
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data['agent_outcome']
    # Execute the tool
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    # Return the output
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

### Define the Graph

In [ ]:
from langgraph.graph import StateGraph, END

In [ ]:

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [ ]:
type(app)

In [ ]:
app.get_graph().print_ascii()

In [ ]:
# invoke the agent
inputs = {"input": "give me a random number and then write in words and make it lower case.",
          "chat_history": [],
          "intermediate_steps":[]}
app.invoke(inputs)